# Banco de dados

Banco de dados é uma estrutura organizada para armazenar, gerenciar e recuperar informações de forma eficiente. Um termo conhecido é o CRUD (create, read, update e delete). Conheceremos agora um banco de dados para testes, chamado Sqlite3.

In [1]:
# importando a biblioteca para fazer operações em banco de dados sqlite3
import sqlite3
# importando a bibliotecas pandas para transformar dados e tabelas do banco em data frames 
import pandas as pd

# CONNECTION

In [2]:
# criaremos a conexão com nosso banco de dados
# se ele não existir, será criado
conn = sqlite3.connect("database.db")

# CREATE TABLE

In [3]:
# Podemos agora criar tabelas
# criaremos a tabela (table) tasks com campos (fields) id, description e done
conn.execute("""
    CREATE TABLE IF NOT EXISTS tasks
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        description TEXT,
        done INTEGER
    )    
""")
# após executar uma operação é necessário salvá-la
conn.commit()

# CREATE

In [4]:
# criaremos agora um registro (row) na tabela tasks
# o registro terá valor 'Estudar seaborn' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar seaborn', 0);    
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [5]:
# TODO

# crie um registro (row) na tabela tasks
# o registro terá valor 'Estudar python' para description e 1 para done.
conn.execute("""
 
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [6]:
# TODO

# crie um registro (row) na tabela tasks
# o registro terá valor 'Entregar livro' para description e 0 para done.
conn.execute("""
 
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

# READ

In [7]:
# podemos agora visualizar todos (*) os registros da (FROM) tabela tasks
results = conn.execute("""
    SELECT *
    FROM tasks
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

(1, 'Estudar seaborn', 0)


In [8]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done=0
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

('Estudar seaborn',)


In [9]:
# TODO

# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0 e description começa com 'Estudar'
results = conn.execute("""

""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

# UPDATE

In [10]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo id = 1 fazendo (SET) done = 1
conn.execute("""
    UPDATE tasks
    SET done=1
    WHERE id=1
""")
# salvando as atualizações
conn.commit()

In [11]:
# TODO

# Atualizar (update) na tabela tasks as linhas (rows) cujo description = "Estudar seaborn" fazendo (SET) description = "Estudar matplotlib"
conn.execute("""

""")
# salvando as atualizações
conn.commit()

# DELETE

In [12]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn.execute("""
    DELETE FROM tasks
    WHERE id=1
""")
conn.commit()

# Pandas query

Quando se fala em tabelas, uma das bibliotecas mais utilizar é a biblioteca pandas. Para nossa conveniência, o método read_sql_query da biblioteca retorna data frames de consultas SQL.

In [13]:
df = pd.read_sql_query(
    """
       SELECT *
       FROM tasks
    """,
    conn
)
df

,id,description,done


# AGRUPAMENTO E AGREGAÇÃO

Existe um paralelo entre os métodos da biblioteca pandas e as queries SQL. Vejamos por exemplo como contar as linhas da tabela tasks onde done = 1

In [14]:
df['done'].value_counts()

Series([], Name: count, dtype: int64)

Podemos fazer diretamente usando SQL:

In [15]:
results = conn.execute("""
    SELECT done, COUNT(*) AS done_tasks
    FROM tasks
    GROUP BY done;      
""").fetchall()

for result in results:
    print(result)

# Encerrando a conexão

In [16]:
conn.close()

# RELACIONAMENTO

Antes de começar você pode apagar o arquivo database.db, vamos recriá-lo. Se decidir por não apagar, mude o nome do arquivo dessa nova conexão.

In [20]:
conn = sqlite3.connect("database.db")

conn.execute("""
    CREATE TABLE categories 
    (
      id    INTEGER PRIMARY KEY, 
      name  TEXT NOT NULL
    );
""")

conn.execute(
    """
        INSERT INTO categories (name)
        VALUES
          ('casa'),
          ('laser'),
          ('faculdade')
    """
)
conn.commit()

A tabela tasks agora terá uma chave estrangeira (FOREIGN KEY) que faz referência ao campo id da tabela categories. Logo estas duas tabelas estão conectadas.

In [21]:
conn.execute("""
    CREATE TABLE tasks 
    (
      id            INTEGER PRIMARY KEY AUTOINCREMENT, 
      description   TEXT NOT NULL,
      done          INTEGER,
      category_id   INTEGER NOT NULL,
      FOREIGN KEY (category_id) REFERENCES categories (id)
    )
""")
conn.commit()

<center><img src="./assets/tasks_categories_relationship.png" width="400px"></center>

# JOIN

Nosso próximo passo é fazer uma querie que junta as duas tabelas anteriores numa só. Antes, vamos inserir alguns dados na tabela tasks.

In [22]:
# insert many
conn.execute(
    """
    INSERT INTO 'tasks' (description, done, category_id)
    VALUES
      ('Assistir Netflix', 1, 2),
      ('Estudar matplotlib', 0, 3),
      ('Estudar seaborn', 0, 3),
      ('Estudar Pandas', 0, 3),
      ('Lavar louça', 0, 1)
    """
)
conn.commit()

Agora sim, juntaremos as tabelas.

In [23]:
tasks_with_categories = conn.execute(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """
).fetchall()

for task in tasks_with_categories:
    print(task)

(5, 'Lavar louça', 0, 'casa')
(2, 'Estudar matplotlib', 0, 'faculdade')
(3, 'Estudar seaborn', 0, 'faculdade')
(4, 'Estudar Pandas', 0, 'faculdade')
(1, 'Assistir Netflix', 1, 'laser')


Usando pandas para ter o resultado como um objeto data frame.

In [24]:
df = pd.read_sql_query(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """,
    conn
)
df

,id,description,done,name
0,5,Lavar louça,0,casa
1,2,Estudar matplotlib,0,faculdade
2,3,Estudar seaborn,0,faculdade
3,4,Estudar Pandas,0,faculdade
4,1,Assistir Netflix,1,laser


# Close connection

In [25]:
conn.close()

# Exercícios

<img src="assets/sqlite-sample-database-color.jpg">

Faça o download do banco SQLite3 no site: https://www.sqlitetutorial.net/sqlite-sample-database/

O arquivo também está na pastas assets, caso não queira fazer o download.

Usando os conhecimentos da aula, resolva os seguinte problemas:

1. Fornecer uma consulta que mostre os clientes (apenas seus nomes completos, ID do cliente e país) que não estão nos EUA.

In [27]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
        SELECT CustomerId, FirstName || ' ' || LastName AS FullName, Country
        FROM customers
        WHERE Country != 'USA'
    """,
    conn
)
conn.close()
df

,CustomerId,FullName,Country
0,1,Luís Gonçalves,Brazil
1,2,Leonie Köhler,Germany
2,3,François Tremblay,Canada
3,4,Bjørn Hansen,Norway
4,5,František Wichterlová,Czech Republic
5,6,Helena Holý,Czech Republic
6,7,Astrid Gruber,Austria
7,8,Daan Peeters,Belgium
8,9,Kara Nielsen,Denmark
9,10,Eduardo Martins,Brazil


2. Fornecer uma consulta que mostre apenas os clientes do Brasil.

In [28]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4
2,11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,+55 (11) 3055-8131,alero@uol.com.br,5
3,12,Roberto,Almeida,Riotur,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,+55 (21) 2271-7000,+55 (21) 2271-7070,roberto.almeida@riotur.gov.br,3
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


3. Fornecer uma consulta que mostre as faturas dos clientes que são do Brasil. A tabela resultante deve mostrar o nome completo do cliente, ID da fatura, data da fatura e país de cobrança.

In [38]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,...,SupportRepId,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,98,1,2010-03-11 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,3.98
1,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,121,1,2010-06-13 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,3.96
2,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,143,1,2010-09-15 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,5.94
3,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,195,1,2011-05-06 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,0.99
4,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,316,1,2012-10-27 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,1.98
5,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,327,1,2012-12-07 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,13.86
6,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,382,1,2013-08-07 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,8.91
7,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,...,4,25,10,2009-04-09 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,8.91
8,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,...,4,154,10,2010-11-14 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,1.98
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,...,4,177,10,2011-02-16 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,3.96


4. Fornecer uma consulta que mostre apenas os funcionários que são agentes de vendas.

In [39]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,EmployeeId,FullName,Title,Email
0,3,Jane Peacock,Sales Support Agent,jane@chinookcorp.com
1,4,Margaret Park,Sales Support Agent,margaret@chinookcorp.com
2,5,Steve Johnson,Sales Support Agent,steve@chinookcorp.com


5. Fornecer uma consulta que mostre uma lista única de países de cobrança da tabela de faturas.

In [40]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,BillingCountry
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Canada
6,Chile
7,Czech Republic
8,Denmark
9,Finland


6. Fornecer uma consulta que mostre as faturas associadas a cada agente de vendas. A tabela resultante deve incluir o nome completo do agente de vendas.

In [41]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,AgentFullName,InvoiceId,InvoiceDate,Total
0,Jane Peacock,6,2009-01-19 00:00:00,0.99
1,Jane Peacock,7,2009-02-01 00:00:00,1.98
2,Jane Peacock,9,2009-02-02 00:00:00,3.96
3,Jane Peacock,10,2009-02-03 00:00:00,5.94
4,Jane Peacock,11,2009-02-06 00:00:00,8.91
...,...,...,...,...
407,Steve Johnson,398,2013-10-21 00:00:00,0.99
408,Steve Johnson,402,2013-11-05 00:00:00,5.94
409,Steve Johnson,404,2013-11-13 00:00:00,25.86
410,Steve Johnson,406,2013-12-04 00:00:00,1.98


É possível então saber quantas vendas e qual valor cada agente fez. Como?

7. Fornecer uma consulta que mostre o valor total da fatura, nome do cliente, país e nome do agente de vendas de todas as faturas e clientes.

In [42]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,InvoiceTotal,CustomerName,Country,AgentName
0,1.98,Julia Barnett,USA,Steve Johnson
1,13.86,Julia Barnett,USA,Steve Johnson
2,8.91,Julia Barnett,USA,Steve Johnson
3,1.98,Julia Barnett,USA,Steve Johnson
4,3.96,Julia Barnett,USA,Steve Johnson
...,...,...,...,...
407,13.86,Fynn Zimmermann,Germany,Jane Peacock
408,14.91,Fynn Zimmermann,Germany,Jane Peacock
409,1.98,Fynn Zimmermann,Germany,Jane Peacock
410,3.96,Fynn Zimmermann,Germany,Jane Peacock


É possível agora saber qual nosso melhor cliente por valor ou quantidade vendida. Como?

8. Quantas faturas houve em 2009 e 2011? Quais são as vendas totais respectivas para cada um desses anos?

In [43]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,Year,InvoiceCount,TotalSales
0,2009,83,449.46
1,2011,83,469.58


As vendas aumentaram, mas não tanto quanto esperávamos!

9. Analisando a tabela invoices, forneça uma consulta que CONTE o número de itens para o ID da fatura 37.

In [49]:

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,InvoiceId,ItemsQuantity
0,37,4


10. Analisando a tabela invoice_items, forneça uma consulta que CONTE o número de itens para cada fatura. DICA: AGRUPAR POR

In [53]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,InvoiceId,ItemCount
0,5,14
1,12,14
2,19,14
3,26,14
4,33,14
...,...,...
407,384,1
408,391,1
409,398,1
410,405,1


11. Fornecer uma consulta que inclua o nome da faixa para cada linha da tabela de itens da fatura.

In [54]:

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,InvoiceLineId,InvoiceId,TrackId,TrackName
0,1,1,2,Balls to the Wall
1,2,1,4,Restless and Wild
2,3,2,6,Put The Finger On You
3,4,2,8,Inject The Venom
4,5,2,10,Evil Walks
...,...,...,...,...
2235,2236,411,3136,Looking For Love
2236,2237,411,3145,Sweet Lady Luck
2237,2238,411,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...
2238,2239,411,3163,Samba pras moças


12. Forneça uma consulta que mostre o nome da música e o artista para cada linha da tabela de itens da fatura.

In [56]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,InvoiceLineId,TrackName,ArtistName
0,1,Balls to the Wall,Accept
1,2,Restless and Wild,Accept
2,3,Put The Finger On You,AC/DC
3,4,Inject The Venom,AC/DC
4,5,Evil Walks,AC/DC
...,...,...,...
2235,2236,Looking For Love,Lenny Kravitz
2236,2237,Sweet Lady Luck,Lenny Kravitz
2237,2238,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,Zeca Pagodinho
2238,2239,Samba pras moças,Zeca Pagodinho


13. Fornecer uma consulta que mostre o número de faturas por país. DICA: AGRUPAR POR

In [57]:
# TODO
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,BillingCountry,InvoiceCount
0,USA,91
1,Canada,56
2,France,35
3,Brazil,35
4,Germany,28
5,United Kingdom,21
6,Portugal,14
7,Czech Republic,14
8,India,13
9,Sweden,7


14. Forneça uma consulta que mostra o número total de música por playlist. O nome e ID das playlists devem aparecer na tabela resultante.

In [60]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,PlaylistId,PlaylistName,TrackCount
0,1,Music,3290
1,8,Music,3290
2,5,90’s Music,1477
3,3,TV Shows,213
4,10,TV Shows,213
5,12,Classical,75
6,11,Brazilian Music,39
7,17,Heavy Metal Classic,26
8,13,Classical 101 - Deep Cuts,25
9,14,Classical 101 - Next Steps,25


15.Forneça uma consulta que mostra todos os nomes das músicas, mas não mostra ids. A tabela resultante deve incluir o nome do álbum, tipo de media e gênero.

In [61]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,TrackName,AlbumTitle,MediaType,Genre
0,"""40""",War,MPEG audio file,Rock
1,"""?""","Lost, Season 2",Protected MPEG-4 video file,TV Shows
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Sir Neville Marriner: A Celebration,Protected AAC audio file,Classical
3,#1 Zero,Out Of Exile,MPEG audio file,Alternative & Punk
4,#9 Dream,Instant Karma: The Amnesty International Campa...,Protected AAC audio file,Pop
...,...,...,...,...
3498,É que Nessa Encarnação Eu Nasci Manga,Axé Bahia 2001,MPEG audio file,Pop
3499,"Étude 1, In C Major - Preludio (Presto) - Liszt",Liszt - 12 Études D'Execution Transcendante,Purchased AAC audio file,Classical
3500,Óculos,Arquivo Os Paralamas Do Sucesso,MPEG audio file,Latin
3501,Óia Eu Aqui De Novo,As Canções de Eu Tu Eles,MPEG audio file,Soundtrack


16. Fornecer uma consulta que mostra o número total de vendas de cada vendedor.

In [62]:

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,SalesAgent,TotalSales
0,Jane Peacock,833.04
1,Margaret Park,775.40
2,Steve Johnson,720.16


17. Qual vendedor (sales agent) mais vendeu em 2009?

In [63]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,SalesAgent,TotalSales
0,Steve Johnson,164.34


18. Qual vendedor (sales agent) mais vendeu considerando toda base de dados?

In [64]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,SalesAgent,TotalSales
0,Jane Peacock,833.04


19. Forneça uma consulta que mostra o número de clientes inscritos para cada vendedor (sales agente).

In [65]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,SalesAgent,CustomerCount
0,Jane Peacock,21
1,Margaret Park,20
2,Steve Johnson,18


20. Forneça uma consulta que mostra o total de vendas por país.

In [67]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,BillingCountry,TotalSales
0,USA,91
1,Canada,56
2,France,35
3,Brazil,35
4,Germany,28
5,United Kingdom,21
6,Portugal,14
7,Czech Republic,14
8,India,13
9,Sweden,7


21. De qual país os clientes mais gastaram?

In [68]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,BillingCountry,TotalSales
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


22. Forneça uma consulta que mostra a faixa mais comprada de 2013.

In [108]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,TrackName,PurchaseCount
0,Black Diamond,1


Será que está certo o resultado? Pense sobre!

23. Forneça uma consulta que mostra os top 3 artistas mais vendidos de toda base de dados.

In [110]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,ArtistName,TotalSold
0,Iron Maiden,140
1,U2,107
2,Metallica,91


24. Forneça uma query que mostra qual o tipo de media mais comprado.

In [114]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """

    """,
    conn
)
conn.close()
df

,MediaType,PurchaseCount
0,MPEG audio file,1976


### Referência para os exercícios:

http://www.berkut.mk.ua/download/pdf/db/task_vesna.pdf